# Handle B-Cell Data with HybridVAE

In [9]:
import sys
sys.path.append("/Users/sarahnarrowedanielsson/Documents/KTH/Exjobb/svci-tools") 
# import hybridvi
import scvi
from scvi.model import HYBRIDVI
# import scvi.model.HYBRIDVI
# import scvi.module as module
from os import path
import scanpy as sc

import math
import anndata

# matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors
plt.style.use('seaborn-whitegrid')
import numpy as np

import pandas as pd

ImportError: cannot import name 'HYBRIDVI' from 'scvi.model' (/opt/homebrew/anaconda3/envs/scvi_tools/lib/python3.8/site-packages/scvi/model/__init__.py)

## Prepare Data 

In [4]:
def concatenate_adatas(list_adata):
    return anndata.AnnData.concatenate(*list_adata,batch_key='batch')

datasets =[
    "SRR11816791",
    "SRR11816792"
]

list_adata=[]
for i in range(len(datasets)):
    fname = "../data/"+datasets[i]+"/filtered_feature_bc_matrix.h5"
    adata = sc.read_10x_h5(fname)
    adata.var_names_make_unique()
    adata.obs["dataset"] = datasets[i]
    adata.obs["tech"] = "v2rna"
    list_adata.append(adata)
    
adata = concatenate_adatas(list_adata)

sc.pp.filter_cells(adata, min_genes=20)  #lower than usual
sc.pp.filter_genes(adata, min_cells=3)

adata.layers["counts"] = adata.X.copy() # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata # freeze the state in `.raw`

scvi.data.setup_anndata(
    adata,
    layer="counts"
)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


INFO     No batch_key inputted, assuming all cells are same batch               
INFO     No label_key inputted, assuming all cells have same label              
INFO     Using data from adata.layers["counts"]                                 
INFO     Computing library size prior per batch                                 
INFO     Successfully registered anndata object containing 9822 cells, 21404    
         vars, 1 batches, 1 labels, and 0 proteins. Also registered 0 extra     
         categorical covariates and 0 extra continuous covariates.              
INFO     Please do not further modify adata until model is trained.             


## Train Model With B-Cell Data

In [10]:
import torch
from datetime import datetime
from datetime import date

today = date.today()
now = datetime.now()
# dd/mm/YY
current_date = today.strftime("%d_%m_%Y")
current_time = now.strftime("%H_%M")
name=current_date+current_time

model = scvi.model.HYBRIDVI(adata)
if (path.exists("../saved_model/")):
    model = torch.load('../saved_model/'+name+'hybridvae.model.pkl')
else:
    model.train()
    torch.save(model,'../saved_model/'+name+'hybridvae.model.pkl')

AttributeError: module 'scvi.model' has no attribute 'HYBRIDVI'